In [1]:
from sklearn import svm
import numpy as np
import random as rand

In [2]:
def classify_point(m, b, x, y):
    expected_value = m * x + b

    if y >= expected_value:
        return 1
    else:
        return -1

In [3]:
def generate_target_function():
    x0 = rand.uniform(-1, 1)
    y0 = rand.uniform(-1, 1)
    x1 = rand.uniform(-1, 1)
    y1 = rand.uniform(-1, 1)

    m_f = (y1 - y0) / (x1 - x0)
    b_f = y0 - m_f * x0
    
    return m_f, b_f

In [4]:
def generate_data(m, b, num_points):
    bias = [1 for _ in range(num_points)]
    x1 = [rand.uniform(-1, 1) for _ in range(num_points)]
    x2 = [rand.uniform(-1, 1) for _ in range(num_points)]

    xn_pre_bias = np.column_stack((x1, x2))
    xn = np.column_stack((bias, xn_pre_bias))
    yn = [classify_point(m, b, x1, x2) for (bias, x1, x2) in xn]

    training_data = np.column_stack((xn, yn))
    
    return training_data

In [6]:
def update_misclassified_points(m_g, b_g, data):
    xn = data[:, np.array([False, True, True, False])]
    f_classification = data[:, -1]
    g_classfication = [classify_point(m_g, b_g, x, y) for (x, y) in xn]
    
    new_misclassified_points = []
    
    for i in range(len(f_classification)):
        if f_classification[i] != g_classfication[i]:
            new_misclassified_points.append(data[i])
    
    return new_misclassified_points

In [7]:
def PLA(w, m_g, b_g, training_data):
    iterations = [w]

    misclassified_points = update_misclassified_points(m_g, b_g, training_data)
    
    while len(misclassified_points) != 0:
        new_w = iterations[-1]
        m_g = -1 * new_w[1] / new_w[2]
        b_g = -1 * new_w[0] / new_w[2]
        
        random_index = np.random.randint(0, len(misclassified_points))
        misclassified_point = misclassified_points[random_index]

        w = np.array(new_w) + misclassified_point[-1] * np.array(misclassified_point[0])
        iterations.append(new_w)
        
        misclassified_points = update_misclassified_points(m_g, b_g, training_data)
    
    return iterations

In [8]:
pla_weights = []
svm_weights = []

In [9]:
from sklearn import linear_model

In [ ]:
data = []

In [10]:
def calc_error(w, f_classification):
    xn = data[:, np.array([True, True, True, False])]
    f_classification = data[:, -1]
    g_classfication = [np.sign(np.dot(w, x)) for x in xn]
    
    counter = 0
    for i in range(len(f_classification)):
        if f_classification[i] != g_classfication[i]:
            counter += 1
    
    return counter / len(xn)

In [11]:
for i in range(1000):
    m, b = generate_target_function()
    data = generate_data(m, b, 10)
    
    while len(np.unique(data[:, -1])) == 1:
        data = generate_data(m, b, 10)

    X = data[:, 0:3]
    y = data[:, 3]
    clflin = linear_model.Perceptron()
    clflin.fit(X, y)
    pla_weights.append(calc_error(clflin.coef_, y))
    
    clf = svm.SVC(kernel='linear', C=1e10)
    clf.fit(X, y)
    svm_weights.append(calc_error(clf.coef_, y))

In [12]:
counter = 0

for pla_weight, svm_weight in zip(pla_weights, svm_weights):
    if pla_weight < svm_weight:
        counter+= 1

counter / 1000

0.622

In [13]:
pla_weights = []
svm_weights = []
num_svm_vectors = []

In [14]:
for i in range(1000):
    m, b = generate_target_function()
    data = generate_data(m, b, 10)
    
    while len(np.unique(data[:, -1])) == 1:
        data = generate_data(m, b, 100)

    X = data[:, 0:3]
    y = data[:, 3]
    clflin = linear_model.Perceptron()
    clflin.fit(X, y)
    pla_weights.append(calc_error(clflin.coef_, y))
    
    clf = svm.SVC(kernel='linear', C=1e10)
    clf.fit(X, y)
    svm_weights.append(calc_error(clf.coef_, y))
    num_svm_vectors.append(len(clf.support_vectors_))

In [15]:
counter = 0

for pla_weight, svm_weight in zip(pla_weights, svm_weights):
    if pla_weight < svm_weight:
        counter+= 1

counter / 1000

0.625

In [16]:
np.mean(num_svm_vectors)

2.875